In [2]:
%cd /content
%pip install -r /content/requirments.txt

/pyenv/versions/3.10.14/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/content

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Setup environment
import os
from dotenv import load_dotenv
from pprint import pprint as pp

load_dotenv()
print(os.environ['ZAPIER_WEBHOOK'])

import jax
jax.devices()

https://hooks.zapier.com/hooks/catch/6996241/23u9vlk/


2024-07-08 02:47:16.426853: E external/xla/xla/stream_executor/rocm/rocm_driver.cc:301] failed call to hipInit: HIP_ERROR_InvalidDevice


[CpuDevice(id=0)]

In [4]:
import keras_nlp
# keras.config.set_floatx("float16")
# Model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
    proto="/tokenizer/gemma_ua_ordered.model"
)

2024-07-08 02:47:23.751797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [ ]:
test_prompts_dev = [
    """Борщ це -""",
    """Крим - це територія"""
]

In [5]:
with open('input_prompts.txt', 'r') as file:
    test_prompts = [line.rstrip('\n') for line in file]

def process_prompts():
    llm_outputs = []
    result_str = ""
    for prompt in test_prompts:
        result_str += f"PROMPT:\"{prompt}\"\n"
        output = gemma_lm.generate(prompt, max_length=96)
        llm_outputs.append(output)
        result_str += f"LLM:\n{output}\n\n"
    return result_str

In [6]:
# Zapier snippet
import requests
from datetime import datetime
webhook_url = os.environ["ZAPIER_WEBHOOK"]
model_metadata = "blank metadata"
body = "blank body"
def post_to_slack():
    requests.post(webhook_url, json={'time': str(datetime.now().isoformat()), 'metadata': model_metadata, 'body': body, 'server': 'AMD mi300x'})

In [ ]:
model_metadata = f"gemma-7b base model vanilla"
body = process_prompts()
post_to_slack()

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
    proto="/tokenizer/gemma_ua_ordered.model"
)
model_metadata = f"gemma-7b base model with custom UA tokenizer"
body = process_prompts()
post_to_slack()

In [ ]:
gemma_lm.load_weights("/ckpts/ckpt_meaninit_full_step_260000_loss_0.99.weights.h5")
model_metadata = f"gemma-7b base model with custom UA tokenizer and embd trained on 1B wiki-ua dataset"
body = process_prompts()
post_to_slack()

In [7]:
def sort_filenames_alphanumerically(filenames):
    # Define a function to extract the step value from the filename
    def extract_step(filename):
        parts = filename.split('_')
        step_index = parts.index('step') + 1
        return int(parts[step_index])
    
    # Sort the list using the step value as the key
    filenames.sort(key=extract_step)

In [8]:
def extract_step(filename):
    parts = filename.split('_')
    step_index = parts.index('step') + 1
    return int(parts[step_index])

In [12]:
folder_path = "/tmp/output/EN_UA_2k_mixed_precision_UA_tokenizer"
file_names = os.listdir(folder_path)
sort_filenames_alphanumerically(file_names)
print(file_names)
percents = [35, 50, 70, 85, 100, 130, 150, 200]
for elem in zip(percents, file_names):
    iter = extract_step(elem[1])
    print(iter, f"{elem[1]}", f"{folder_path}{elem[1]}")
    
    # gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
    gemma_lm.load_weights(f"{folder_path}/{elem[1]}")
    # gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
    #     proto="/tokenizer/gemma_ua_ordered.model"
    # )
    model_metadata = f"ckpt iter {iter}/132000 EN_UA dataset, custom UA tokenizer mixed precision"
    body = process_prompts()
    post_to_slack()


['ckpt_step_30000_loss_0.923_accuracy_0.137.weights.h5', 'ckpt_step_38000_loss_0.878_accuracy_0.142.weights.h5']
30000 ckpt_step_30000_loss_0.923_accuracy_0.137.weights.h5 /tmp/output/EN_UA_2k_mixed_precision_UA_tokenizerckpt_step_30000_loss_0.923_accuracy_0.137.weights.h5
38000 ckpt_step_38000_loss_0.878_accuracy_0.142.weights.h5 /tmp/output/EN_UA_2k_mixed_precision_UA_tokenizerckpt_step_38000_loss_0.878_accuracy_0.142.weights.h5


In [11]:
folder_path = "/tmp/output/EN_UA_2k_mixed_precision_UA_tokenizer_embd"
file_names = os.listdir(folder_path)
sort_filenames_alphanumerically(file_names)
print(file_names)
percents = [35, 50, 70, 85, 100, 130, 150, 200]
for elem in zip(percents, file_names):
    iter = extract_step(elem[1])
    print(iter, f"{elem[1]}", f"{folder_path}{elem[1]}")
    
    # gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
    gemma_lm.load_weights(f"{folder_path}/{elem[1]}")
    # gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
    #     proto="/tokenizer/gemma_ua_ordered.model"
    # )
    model_metadata = f"ckpt iter {iter}/132000 EN_UA dataset, custom UA tokenizer mixed precision, embd layer pretraining"
    body = process_prompts()
    post_to_slack()

['ckpt_step_26000_loss_0.600_accuracy_0.172.weights.h5', 'ckpt_step_32000_loss_0.595_accuracy_0.172.weights.h5']
26000 ckpt_step_26000_loss_0.600_accuracy_0.172.weights.h5 /tmp/output/EN_UA_2k_mixed_precision_UA_tokenizer_embdckpt_step_26000_loss_0.600_accuracy_0.172.weights.h5
32000 ckpt_step_32000_loss_0.595_accuracy_0.172.weights.h5 /tmp/output/EN_UA_2k_mixed_precision_UA_tokenizer_embdckpt_step_32000_loss_0.595_accuracy_0.172.weights.h5
